# Feature selection with regression lab

You have been exploring feature selection, regularization, and evaluation metrics with classification this week (and last week).

It's time to return to a regression problem.

In this lab you will be working with a dataset on **reservoir levels in California**.

---

### Data summary

The dataset contains these columns:

    id : the reservoir string id
    date : the string date of measurement
    month : the month of measurement
    year : the year of measurement
    reservoir_volume : the current volume of the reservoir
    dam : the dam that supplies water to the reservoir
    lake : a lake associated with the reservoir
    stream : a stream associated with the reservoir
    capacity : the total capacity of the reservoir
    elevation : the string for elevation of the reservoir
    river_basin : string code for basin of the river associated with the reservoir
    county : the string county code that the reservoir is in
    hydrologic_area : another location string identifier
    nearby_city : the city that the reservoir is located near
    latitude : the exact latitude of the reservoir
    longitude : the exact longitude of the reservoir
    operator : the governmental organization that manages the reservoir

---

### Goal of the lab:

Your primary goal is to **predict reservoir levels in 2015 using data from prior years**. Your primary evaluation metric will be **RMSE**, but feel free to try/use others as well.

You have learned may useful skills to tackle this!

- **Data munging/cleaning**
- **EDA**
- **Feature engineering**
- **Feature selection**
- **Regularization/gridsearching**
- **Cross-validation**

Use whatever you can to get the best predictive power! Godspeed.

---

### I WILL BE DOING THE LAB AT THE SAME TIME

**You are competing with me in this lab to get a better RMSE.**

YES, YES, I know it's "unfair" or whatever. However, I will be sharing my progress on my screen as I go through, so you have an advantage in that sense, since you can see what I'm doing as I go.

#### WORK TOGETHER!!!

Good Luck!

---

In [1]:
import pandas as pd
import numpy as np

In [2]:
res = pd.read_csv('./dataset/reservoir_levels.csv')

In [3]:
# look at data head
res.head()

,date,month,year,id,reservoir_volume,dam,lake,stream,capacity,elevation,river_basin,county,hydrologic_area,nearby_city,latitude,longitude,operator
0,2000-02,2,2000,APN,8891.0,Alpine,Alpine Lake,Lagunitas Creek,8900,654' ft,COAST-MARIN,MARIN,SAN FRANCISCO BAY,STINSON BEACH,37.94,-122.637,Marin Municipal Water District
1,2000-03,3,2000,APN,8891.0,Alpine,Alpine Lake,Lagunitas Creek,8900,654' ft,COAST-MARIN,MARIN,SAN FRANCISCO BAY,STINSON BEACH,37.94,-122.637,Marin Municipal Water District
2,2000-04,4,2000,APN,8891.0,Alpine,Alpine Lake,Lagunitas Creek,8900,654' ft,COAST-MARIN,MARIN,SAN FRANCISCO BAY,STINSON BEACH,37.94,-122.637,Marin Municipal Water District
3,2000-05,5,2000,APN,8891.0,Alpine,Alpine Lake,Lagunitas Creek,8900,654' ft,COAST-MARIN,MARIN,SAN FRANCISCO BAY,STINSON BEACH,37.94,-122.637,Marin Municipal Water District
4,2000-06,6,2000,APN,8250.0,Alpine,Alpine Lake,Lagunitas Creek,8900,654' ft,COAST-MARIN,MARIN,SAN FRANCISCO BAY,STINSON BEACH,37.94,-122.637,Marin Municipal Water District


In [4]:
res.elevation.values[0:5]

array(["654' ft", "654' ft", "654' ft", "654' ft", "654' ft"], dtype=object)

In [5]:
def try_convert_elevation(x):
    try:
        return float(x.split("'")[0])
    except:
        return np.nan
    
tmp_elevation = res.elevation.map(try_convert_elevation)

In [6]:
sum(tmp_elevation.isnull())

366

In [7]:
res.shape

(34038, 17)

In [8]:
res[tmp_elevation.isnull()].head(10)

,date,month,year,id,reservoir_volume,dam,lake,stream,capacity,elevation,river_basin,county,hydrologic_area,nearby_city,latitude,longitude,operator
10431,2000-02,2,2000,GBR,68758.0,Gerber,Gerber Lake,Klamath River,94300,ft,KLAMATH R,SISKIYOU,NORTH COAST,KLAMATH FALLS,42.205,-121.13,US Bureau of Reclamation
10432,2000-03,3,2000,GBR,83538.0,Gerber,Gerber Lake,Klamath River,94300,ft,KLAMATH R,SISKIYOU,NORTH COAST,KLAMATH FALLS,42.205,-121.13,US Bureau of Reclamation
10433,2000-04,4,2000,GBR,92854.0,Gerber,Gerber Lake,Klamath River,94300,ft,KLAMATH R,SISKIYOU,NORTH COAST,KLAMATH FALLS,42.205,-121.13,US Bureau of Reclamation
10434,2000-05,5,2000,GBR,91410.0,Gerber,Gerber Lake,Klamath River,94300,ft,KLAMATH R,SISKIYOU,NORTH COAST,KLAMATH FALLS,42.205,-121.13,US Bureau of Reclamation
10435,2000-06,6,2000,GBR,76998.0,Gerber,Gerber Lake,Klamath River,94300,ft,KLAMATH R,SISKIYOU,NORTH COAST,KLAMATH FALLS,42.205,-121.13,US Bureau of Reclamation
10436,2000-07,7,2000,GBR,67262.0,Gerber,Gerber Lake,Klamath River,94300,ft,KLAMATH R,SISKIYOU,NORTH COAST,KLAMATH FALLS,42.205,-121.13,US Bureau of Reclamation
10437,2000-08,8,2000,GBR,57364.0,Gerber,Gerber Lake,Klamath River,94300,ft,KLAMATH R,SISKIYOU,NORTH COAST,KLAMATH FALLS,42.205,-121.13,US Bureau of Reclamation
10438,2000-09,9,2000,GBR,52570.0,Gerber,Gerber Lake,Klamath River,94300,ft,KLAMATH R,SISKIYOU,NORTH COAST,KLAMATH FALLS,42.205,-121.13,US Bureau of Reclamation
10439,2000-10,10,2000,GBR,51547.0,Gerber,Gerber Lake,Klamath River,94300,ft,KLAMATH R,SISKIYOU,NORTH COAST,KLAMATH FALLS,42.205,-121.13,US Bureau of Reclamation
10440,2000-11,11,2000,GBR,51733.0,Gerber,Gerber Lake,Klamath River,94300,ft,KLAMATH R,SISKIYOU,NORTH COAST,KLAMATH FALLS,42.205,-121.13,US Bureau of Reclamation


In [9]:
res['elevation'] = tmp_elevation

In [10]:
elevation_median = np.nanmedian(res.elevation)

In [11]:
print elevation_median

1474.5


In [12]:
res.ix[res.elevation.isnull(), 'elevation'] = elevation_median

In [13]:
sum(res.elevation.isnull())

0

In [14]:
res['pct_capacity'] = res.reservoir_volume/res.capacity

In [15]:
res[res.pct_capacity > 1].head(5)

,date,month,year,id,reservoir_volume,dam,lake,stream,capacity,elevation,river_basin,county,hydrologic_area,nearby_city,latitude,longitude,operator,pct_capacity
185,2000-04,4,2000,ANT,23428.0,Antelope,Antelope,Indian Creek,22600,4960.0,FEATHER R,PLUMAS,SACRAMENTO RIVER,SUSANVILLE,40.18,-120.607,CA Dept of Water Resources/O & M,1.036637
186,2000-05,5,2000,ANT,22853.0,Antelope,Antelope,Indian Creek,22600,4960.0,FEATHER R,PLUMAS,SACRAMENTO RIVER,SUSANVILLE,40.18,-120.607,CA Dept of Water Resources/O & M,1.011195
209,2002-04,4,2002,ANT,23210.0,Antelope,Antelope,Indian Creek,22600,4960.0,FEATHER R,PLUMAS,SACRAMENTO RIVER,SUSANVILLE,40.18,-120.607,CA Dept of Water Resources/O & M,1.026991
210,2002-05,5,2002,ANT,22900.0,Antelope,Antelope,Indian Creek,22600,4960.0,FEATHER R,PLUMAS,SACRAMENTO RIVER,SUSANVILLE,40.18,-120.607,CA Dept of Water Resources/O & M,1.013274
221,2003-04,4,2003,ANT,23239.0,Antelope,Antelope,Indian Creek,22600,4960.0,FEATHER R,PLUMAS,SACRAMENTO RIVER,SUSANVILLE,40.18,-120.607,CA Dept of Water Resources/O & M,1.028274


In [16]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [17]:
res[['longitude','latitude','pct_capacity']].corr()

,longitude,latitude,pct_capacity
longitude,1.000000,-0.738773,-0.156006
latitude,-0.738773,1.000000,0.090362
pct_capacity,-0.156006,0.090362,1.000000


In [18]:
res.columns

Index([u'date', u'month', u'year', u'id', u'reservoir_volume', u'dam', u'lake',
       u'stream', u'capacity', u'elevation', u'river_basin', u'county',
       u'hydrologic_area', u'nearby_city', u'latitude', u'longitude',
       u'operator', u'pct_capacity'],
      dtype='object')

In [19]:
import patsy
import statsmodels.formula.api as smf

In [20]:
res['elevation_n'] = (res.elevation - res.elevation.mean())/res.elevation.std()
res['capacity_c'] = res.capacity - res.capacity.mean()

In [21]:
print smf.ols(formula='pct_capacity ~ C(year) + C(month)', data=res).fit().summary()

                            OLS Regression Results                            
Dep. Variable:           pct_capacity   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     162.6
Date:                Thu, 12 May 2016   Prob (F-statistic):               0.00
Time:                        12:47:33   Log-Likelihood:                -1810.5
No. Observations:               31839   AIC:                             3675.
Df Residuals:                   31812   BIC:                             3901.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           0.6354      0.008     

In [22]:
formula = '''
reservoir_volume ~ capacity_c + elevation_n + C(id) + C(dam) + C(lake) +
C(stream) + C(river_basin) + C(county) + C(hydrologic_area) + C(nearby_city) +
C(operator) + C(month) + C(year) - 1'''



In [48]:
Y, X = patsy.dmatrices(formula, data=res)

X = pd.DataFrame(X, columns=X.design_info.column_names)
Y = pd.DataFrame(Y, columns=['is_spam'])

Y2015_mask = X['C(year)[T.2015]'] == 1
print sum(Y2015_mask)

Xtrain, Xtest = X[~Y2015_mask].values, X[Y2015_mask].values
Ytrain, Ytest = Y[~Y2015_mask].values, Y[Y2015_mask].values

505


In [52]:
print Y.shape, X.shape
print Ytrain.shape, Xtrain.shape
print Ytrain[0:3], Ytest[0:3]

(30928, 1) (30928, 1018)
(30423, 1) (30423, 1018)
[[ 8891.]
 [ 8891.]
 [ 8891.]] [[ 8765.]
 [ 8891.]
 [ 8832.]]


In [50]:
Xtrain_c = Xtrain - np.mean(Xtrain, axis=0)
Xtest_c = Xtest - np.mean(Xtrain, axis=0)

In [40]:
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score

In [51]:
print Xtrain_c.shape, Ytrain.shape, Xtrain.shape
lm = LinearRegression()
lm.fit(Xtrain_c, Ytrain)
print lm.score(Xtest_c, Ytest)

(30423, 1018) (30423, 1) (30423, 1018)
0.865090597856


In [53]:
from sklearn.linear_model import RandomizedLasso

In [62]:
rl = RandomizedLasso(alpha='bic', sample_fraction=0.7, n_resampling=1000, verbose=2)
rl.fit(Xtrain_c, Ytrain)

........................................

//anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Done  40 tasks       | elapsed:   37.6s


....................................................................................................................................................................................................................................................................................................................................

[Parallel(n_jobs=1)]: Done 161 tasks       | elapsed:  2.6min
[Parallel(n_jobs=1)]: Done 364 tasks       | elapsed:  5.7min


............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

[Parallel(n_jobs=1)]: Done 647 tasks       | elapsed: 10.0min
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed: 15.4min finished


RandomizedLasso(alpha='bic', eps=2.2204460492503131e-16, fit_intercept=True,
        max_iter=500, memory=Memory(cachedir=None), n_jobs=1,
        n_resampling=1000, normalize=True, pre_dispatch='3*n_jobs',
        precompute='auto', random_state=None, sample_fraction=0.7,
        scaling=0.5, selection_threshold=0.25, verbose=2)

In [63]:
fi_scores = pd.DataFrame({'feature':X.columns,
                          'feat_imp':rl.scores_})

In [68]:
fi_scores.sort_values(by='feat_imp', ascending=False, inplace=True)

print sum(fi_scores.feat_imp != 0)

206


In [69]:
fi_scores

,feat_imp,feature
1016,0.987,capacity_c
146,0.604,C(id)[SHA]
1014,0.583,C(year)[T.2014]
123,0.570,C(id)[ORO]
569,0.536,C(stream)[T.Colorado River]
30,0.530,C(id)[CLE]
1001,0.510,C(year)[T.2001]
121,0.496,C(id)[NML]
1008,0.488,C(year)[T.2008]
47,0.482,C(id)[DNP]


In [71]:
imp_cols = fi_scores[fi_scores.feat_imp > 0].feature.values

Xtrain_sub, Xtest_sub = X.ix[~Y2015_mask, imp_cols].values, X.ix[Y2015_mask, imp_cols].values
Xtrain_sub_c = Xtrain_sub - np.mean(Xtrain_sub, axis=0)
Xtest_sub_c = Xtest_sub - np.mean(Xtrain_sub, axis=0)

In [72]:
print Xtrain_sub.shape, Xtest_sub.shape

(30423, 206) (505, 206)


In [73]:
from sklearn.linear_model import ElasticNetCV

In [81]:
enet_cv = ElasticNetCV(l1_ratio=np.linspace(0.,1.,30),
                       n_alphas=200, cv=5, verbose=1)

enet_cv.fit(Xtrain_c, np.ravel(Ytrain))

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
       l1_ratio=array([ 0.     ,  0.03448,  0.06897,  0.10345,  0.13793,  0.17241,
        0.2069 ,  0.24138,  0.27586,  0.31034,  0.34483,  0.37931,
        0.41379,  0.44828,  0.48276,  0.51724,  0.55172,  0.58621,
        0.62069,  0.65517,  0.68966,  0.72414,  0.75862,  0.7931 ,
        0.82759,  0.86207,  0.89655,  0.93103,  0.96552,  1.     ]),
       max_iter=1000, n_alphas=200, n_jobs=1, normalize=False,
       positive=False, precompute='auto', random_state=None,
       selection='cyclic', tol=0.0001, verbose=1)

In [82]:
print enet_cv.alpha_
print enet_cv.l1_ratio_

54817325299.4
0.103448275862


In [83]:
from sklearn.linear_model import ElasticNet

In [84]:
enet = ElasticNet(alpha=enet_cv.alpha_, l1_ratio=enet_cv.l1_ratio_)
enet_scores = cross_val_score(enet, Xtrain_c, np.ravel(Ytrain), cv=5)
print enet_scores
print np.mean(enet_scores)

[ 0.81643763  0.94153735  0.81974675  0.84639086  0.89095637]
0.863013793154
